# Preprocessing Altimetry data

In [ ]:
import os
from rasterio.merge import merge
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import rioxarray
import geopandas as gpd
import pandas as pd
import xarray as xr
from shapely.geometry import box

from rioxarray.merge import merge_arrays


import rioxarray
import xarray as xr



import pandas as pd
from shapely.geometry import box

In [ ]:
# Load the shapefile

dem_dir = "data/raw/other_data/altimetri"
shp_file = "data/raw/catchment_statkraft/catchment.shp"
catchment = gpd.read_file(shp_file)

catchment = catchment.to_crs("EPSG:25832")

#### Check the data

In [ ]:

# Replace with your actual file path
file_path = "data/raw/other_data/altimetri/6805_1_10m_z32.tif"

# Load the DEM
dem = rioxarray.open_rasterio(file_path, masked=True)

# Plot it
dem.plot(cmap="terrain", figsize=(10, 6));

### Concatenate the .tif files and clip them to the catchment

In [ ]:
# List all .tif files in the folder
dem_files = [f for f in os.listdir(dem_dir) if f.endswith(".tif")]

elevation_dfs = []
dem_clipped_list = []

for fname in dem_files:
    fpath = os.path.join(dem_dir, fname)
    dem = rioxarray.open_rasterio(fpath, masked=True)

    # Check for intersection
    left, bottom, right, top = dem.rio.bounds()
    dem_box = gpd.GeoDataFrame(geometry=[box(left, bottom, right, top)], crs=dem.rio.crs)
    
    if not dem_box.intersects(catchment.union_all()).any():
        continue  # Skip if it doesn't overlap

    # Clip and store
    dem_clipped = dem.rio.clip(catchment.geometry, catchment.crs)
    dem_clipped_list.append(dem_clipped)

    # Flatten to DataFrame
    df = dem_clipped.squeeze().to_dataframe(name="elevation").reset_index()
    df = df.dropna(subset=["elevation"])
    elevation_dfs.append(df)

# Sort coordinates for each clipped DEM
dem_clipped_list_sorted = [
    da.sortby(["y", "x"]) for da in dem_clipped_list
]

# Merge them


# Merge all DEMs and save
if dem_clipped_list:
    merged_dem = merge_arrays(dem_clipped_list)
    merged_dem.rio.to_raster("data/processed_w_notebooks/catchment_terrain_dem.tif")
    print("Saved raster: catchment_terrain_dem.tif")

    elevation_df = pd.concat(elevation_dfs, ignore_index=True)

#### Create an elevation dataframe to compare with errors

In [ ]:
# Convert elevation_df to a GeoDataFrame using projected CRS (UTM Zone 32N)
gdf = gpd.GeoDataFrame(
    elevation_df,
    geometry=gpd.points_from_xy(elevation_df['x'], elevation_df['y']),
    crs="EPSG:25832"
)

# Reproject to WGS84 (lat/lon)
gdf = gdf.to_crs("EPSG:4326")

# Extract latitude and longitude into new columns
gdf['latitude'] = gdf.geometry.y
gdf['longitude'] = gdf.geometry.x

# (Optional) Drop geometry column if you don't need it
gdf = gdf.drop(columns='geometry')

# Save or keep as DataFrame
elevation_df_with_latlon = gdf

elevation_df_with_latlon.to_csv("data/processed_w_notebooks/catchment_elevation.csv", index=False)

print("Saved CSV: catchment_elevation.csv")

### Plot the catchment with terrain

In [ ]:
# Load terrain raster
dem = rioxarray.open_rasterio("catchment_terrain_dem.tif", masked=True)

# Load catchment shapefile
catchment = gpd.read_file(shp_file).to_crs(dem.rio.crs)

# Plot
fig, ax = plt.subplots(figsize=(10, 10))

# Plot DEM
dem.squeeze().plot(
    ax=ax,
    cmap='terrain',
    cbar_kwargs={'label': 'Elevation (m)'}
)

# Overlay catchment boundary
catchment.boundary.plot(ax=ax, edgecolor='red', linewidth=2)

# Formatting
ax.set_title("Catchment Area over Terrain Elevation", fontsize=14)
ax.set_xlabel("X")
ax.set_ylabel("Y")
plt.show();
